In [127]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
transport_costn = np.array([[5, 8, 6], [7, 4, 9], [6, 7, 5]])
facnum = 3
factory_locations = ["Factory" + str(i) for i in range(1, facnum + 1)]
customer_regions = ["Customer" + str(i) for i in range(1, facnum + 1)]

transport_costs = {
    ("Customer1", "Factory1"): 5, ("Customer1", "Factory2"): 8, ("Customer1", "Factory3"): 6,
    ("Customer2", "Factory1"): 7, ("Customer2", "Factory2"): 4, ("Customer2", "Factory3"): 9,
    ("Customer3", "Factory1"): 6, ("Customer3", "Factory2"): 7, ("Customer3", "Factory3"): 5
}
customer_demands = {"Customer1": 800, "Customer2": 1200, "Customer3": 1000}

fixed_costs = {"Factory1": 10000, "Factory2": 15000, "Factory3": 12000}
fcn = np.array([10000, 15000, 12000])
cn = np.array([800, 1200, 1000])

In [128]:

def define_master_problem_model(initial_constraints=None):
    """定義主問題模型"""
    master_model = gp.Model("MasterProblem_FacilityLocation")

    # y 變數：是否開設工廠
    y = master_model.addVars(factory_locations, vtype=GRB.BINARY, name="y")

    # theta 變數：子問題的成本下界
    theta = master_model.addVar(name="theta")

    # 目標函數：最小化固定成本 + 子問題成本下界
    master_model.setObjective(gp.quicksum(fixed_costs[j] * y[j] for j in factory_locations) + theta, GRB.MINIMIZE)

    # 初始約束 (可以添加初始割，這裡先為空)
    if initial_constraints:
        for constr in initial_constraints:
            master_model.addConstr(constr)

    master_model._y_vars = y
    master_model._theta = theta
    return master_model


def define_subproblem_model(y_current):
    """定義子問題模型 (對於給定的 y 值)"""
    subproblem_model = gp.Model("Subproblem_FacilityLocation")

    # x 變數：運輸量
    x = subproblem_model.addVars(customer_regions, factory_locations, vtype=GRB.CONTINUOUS, name="x")

    # 目標函數：最小化運輸成本
    subproblem_model.setObjective(
        gp.quicksum(transport_costs[i, j] * x[i, j]
                    for i in customer_regions for j in factory_locations),
        GRB.MINIMIZE)

    # 需求滿足約束
    for i in customer_regions:
        subproblem_model.addConstr(gp.quicksum(x[i, j] for j in factory_locations) == customer_demands[i], name=f"Demand_{i}")

    # 運輸量與工廠開設關聯 (在這個簡化版本中，其實由需求約束和非負變數隱含約束了，更複雜版本需要顯式容量約束)
    # 在這裡，我們假設如果 y_j = 0，則所有 x_ij 都必須為 0。 但由於需求必須滿足，如果我們強制 y_j=0， 則模型會自動處理 (如果沒有其他工廠可以滿足需求，就會不可行)
    print(y_current)
    for j in factory_locations:
        if y_current[j] == 0:
            for i in customer_regions:
                subproblem_model.addConstr(x[i, j] == 0, name=f"NoShip_{i}_{j}") # 實際上，在沒有容量約束的簡化版中，這個約束不是嚴格必要，但加上更符合邏輯


    subproblem_model._x_vars = x
    subproblem_model._y_current_val = y_current # 保存 y_current 值，方便割生成函數使用
    return subproblem_model




def solvefarkas(master_model_x):
    try:
        farkas_model = gp.Model("Farkas_Subproblem")
        farkas_model.Params.OutputFlag = 0  # 静默模式，不输出求解器日志
        print("master_model_x:", master_model_x) # 用于调试，检查 master_model_x 字典
        master_model_x = np.array([master_model_x[j] for j in factory_locations]) # 将字典转换为数组
        W_matrix = transport_costn.T  # 假设 transport_costs 的转置是子问题的 W 矩阵
        h_minus_Tx = cn - fcn @ master_model_x #  h - Tx 项, 示例

        # 定义 Farkas 问题的变量 u，需要是非负的
        u = farkas_model.addMVar(shape=cn.shape[0], lb=0, name="u") # u 的维度应与 customer_demands 的维度一致
        print(W_matrix, h_minus_Tx)
        # 添加 Farkas 引理的约束条件
        farkas_model.addConstr(W_matrix @ u >= 0, "farkas_constr1") #  W^T * u >= 0
        farkas_model.addConstr((h_minus_Tx) @ u <= -1e-4, "farkas_constr2") # (h - Tx)^T * u <= -epsilon，用 <= -1e-6 近似 < 0

        # 设置目标函数 (这里设置为最小化 0，因为我们只关心可行性)
        farkas_model.setObjective(0, gp.GRB.MINIMIZE)

        # 求解 Farkas 模型
        farkas_model.optimize()

        if farkas_model.status == gp.GRB.OPTIMAL or farkas_model.status == gp.GRB.SUBOPTIMAL:
            return u.X  # 返回 u 的解向量
        else:
            print("Farkas 子问题不可行，这在理论上不应发生，请检查模型构建。")
            return None # 理论上不应该发生

    except gp.GurobiError as e:
        print("Gurobi 错误 in solvefarkas: " + str(e.errno) + ": " + str(e))
        return None




def generate_farkas_cut(master_model, master_model_x, u):
    """
    生成 Farkas 割平面，并将其添加到主问题模型中。

    参数:
        master_model: 主问题 Gurobi 模型对象
        master_model_x: 主问题模型中的 y 变量字典 (例如 master_model._y_vars)
                         字典的键是工厂位置名称，值是对应的 Gurobi 变量对象
        u: solvefarkas 函数返回的 u 向量 (numpy 数组)

    返回:
        None (直接修改 master_model)
    """
    if u is None:
        print("无法生成 Farkas 割，因为没有可用的 u 向量。")
        return

    cut_expr = gp.LinExpr() # 创建一个线性表达式
    #  假设 fixed_costs 对应于 T 矩阵，你需要构建 (T^T * u)^T * x 对应的表达式

    print("主问题 y 变量字典 (master_model_x):", master_model_x) # 打印 master_model_x 字典用于调试

    # *** 正确构建 Farkas 割的线性表达式 (适应字典 master_model_x) ***
    # 遍历 factory_locations 列表 (或直接遍历 master_model_x.keys() 如果 factory_locations 和字典键一致)
    for i, factory_name in enumerate(factory_locations): # 使用 factory_locations 列表来控制工厂顺序
        if factory_name in master_model_x: # 确保工厂名称在 master_model_x 字典中存在
            y_var = master_model_x[factory_name] # 通过工厂名称从字典中获取对应的 Gurobi 变量对象
            for j in range(u.shape[0]): # 假设 u 和 fixed_costs 的维度匹配
                cut_expr += (fcn[j] * u[j]) * y_var # 使用 y_var (Gurobi 变量对象)
        else:
            print(f"警告: 工厂位置 '{factory_name}' 不在 master_model_x 字典中。请检查字典键和 factory_locations 列表是否一致。")
            return # 如果工厂位置名称不一致，可能需要停止生成割平面并进行错误处理
        

    rhs_val = np.sum(cn * u) #  示例，计算 h^T * u 的值 (右侧项)

    master_model.addConstr(cut_expr <= rhs_val, "farkas_cut") # 将 Farkas 割添加到主问题模型

    print("Farkas 割已添加到主问题模型。")




    
    


def generate_optimality_cut(subproblem_model, master_model, y_vars, y_current):
    """生成最優性割 (修改為直接添加約束，不需返回)"""
    # 獲取對偶變數 (需求約束的對偶變數)
    demand_duals = {i: subproblem_model.getConstrByName(f"Demand_{i}").getAttr(GRB.Attr.Pi) for i in customer_regions}

    # 構建最優性割的表達式
    cut_expr = master_model._theta >= gp.quicksum(demand_duals[i] * customer_demands[i] for i in customer_regions)

    # **修改：直接將約束添加到主問題模型**
    optimality_cut = master_model.addConstr(cut_expr, name="OptimalityCut")
    # return optimality_cut # 原本返回約束物件
    return None # 修改為返回 None (或保持返回 optimality_cut 也可以)
def generate_feasibility_cut(subproblem_model, master_model, y_vars, y_current):
    """生成可行性割 (修改為直接添加約束，不需返回)"""

    # 簡單可行性割： 排除當前 y 組合 (確保主問題下次迭代選擇不同的 y)
    cut_expr = gp.quicksum(y_vars[j] for j in factory_locations if y_current[j] == 1) - \
                    gp.quicksum(y_vars[j] for j in factory_locations if y_current[j] == 0) <= \
                    len([j for j in factory_locations if y_current[j] == 1]) - 1

    # **修改：直接將約束添加到主問題模型**
    feasibility_cut = master_model.addConstr(cut_expr, name="FeasibilityCut")
    # return feasibility_cut # 原本返回約束物件
    return None # 修改為返回 None (或保持返回 feasibility_cut 也可以)

def benders_decomposition(master_problem_model_func, subproblem_model_func,
                           initial_master_constraints=[],
                           max_iterations=100, convergence_tolerance=1e-6):
    """
    使用 Benders 分解求解問題 (直接添加割版本，不使用 cbLazy)。
    """

    master_model = master_problem_model_func(initial_master_constraints)
    y_vars = master_model._y_vars
    master_obj_val = -float('inf')
    upper_bound = float('inf')
    x_val = None
    y_val = None

    for iteration in range(max_iterations):
        master_model.optimize()
        if master_model.status != GRB.OPTIMAL:
            print(f"主問題求解失敗，狀態碼: {master_model.status}")
            return None, None, None, None, None, None

        current_master_obj_val = master_model.ObjVal
        y_current = {j: y_vars[j].X for j in factory_locations}
        master_obj_val = max(master_obj_val, current_master_obj_val)

        subproblem_model = subproblem_model_func(y_current)
        subproblem_model.optimize()

        if subproblem_model.status == GRB.OPTIMAL:
            # 子問題可行且有界，生成最優性割
            sub_obj_val = subproblem_model.ObjVal
            upper_bound = min(upper_bound, current_master_obj_val + sub_obj_val)
            x_val = {var.VarName: var.X for var in subproblem_model.getVars() if var not in y_vars}
            y_val = y_current

            # 直接添加最優性割到主問題，不再使用 cbLazy
            optimality_cut = generate_optimality_cut(subproblem_model, master_model, y_vars, y_current)
            if optimality_cut:
                master_model.addConstr(optimality_cut, name="OptimalityCut")


        elif subproblem_model.status == GRB.INFEASIBLE:
            # print("  子問題不可行，生成 Farkas Cut")
            # print(np.array(y_current.values()))
            # u_solution = solvefarkas(y_current) # 求解 Farkas 子問題，需要傳遞 y 的解值 (用於建構 Farkas 問題)
            
            # if u_solution is not None:
            #     generate_farkas_cut(master_model, y_vars, u_solution) # 生成 Farkas Cut 並添加到主問題，需要傳遞主問題模型、y 變數 (Gurobi 物件) 和 u 向量
            # else:
            #     print("  警告：未能找到 Farkas 解，Benders 分解可能無法正確收斂。")
            #     break #  或採取其他錯誤處理措施
            # master_model.Params.DualReductions = 0 #  在添加 Farkas Cut 後，建議關閉 DualReductions
            # print(f"  添加 Farkas Cut")
    
    
            feasibility_cut = generate_feasibility_cut(subproblem_model, master_model, y_vars, y_current)
            if feasibility_cut:
                master_model.addConstr(feasibility_cut, name="FeasibilityCut")

        else:
            print(f"子問題求解異常，狀態碼: {subproblem_model.status}")
            return None, None, None, None, None, None

        gap = upper_bound - master_obj_val
        print(f"迭代 {iteration+1}, 下界: {master_obj_val:.4f}, 上界: {upper_bound:.4f}, Gap: {gap:.4f}")

        if gap < convergence_tolerance:
            print("達到收斂!")
            break

    return master_model, subproblem_model, y_val, x_val, master_obj_val, upper_bound




In [129]:
master_problem_func = define_master_problem_model
subproblem_func = define_subproblem_model

initial_constraints = [] # 初始約束為空

master_model, subproblem_model, y_val, x_val, lower_bound, upper_bound = \
    benders_decomposition(master_problem_func, subproblem_func, initial_constraints, max_iterations=20, convergence_tolerance=1e-4)

if master_model:
    print("\nBenders 分解完成!")
    print(f"最優下界: {lower_bound:.4f}")
    print(f"最優上界: {upper_bound:.4f}")
    print("選擇開設的工廠 (y 變數的最優值):", {j: val for j, val in y_val.items() if val > 0.5}) # 輸出開設的工廠
    if x_val:
        print("運輸方案 (x 變數的最優值):")
        for i in customer_regions:
            for j in factory_locations:
                if x_val.get(f'x[{i},{j}]', 0) > 1e-6: # 避免輸出接近 0 的值
                    print(f"  從 {j} 到 {i}: {x_val[f'x[{i},{j}]']:.2f}")
    else:
        print("子問題不可行，沒有 x 的最優值")

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 0 rows, 4 columns and 0 nonzeros
Model fingerprint: 0x2043c49a
Variable types: 1 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
{'Factory1': -0.0, 'Factory2': -0.0, 'Factory3': -0.0}
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruc

In [130]:
fixed_costs

{'Factory1': 10000, 'Factory2': 15000, 'Factory3': 12000}

In [131]:
transport_costs

{('Customer1', 'Factory1'): 5,
 ('Customer1', 'Factory2'): 8,
 ('Customer1', 'Factory3'): 6,
 ('Customer2', 'Factory1'): 7,
 ('Customer2', 'Factory2'): 4,
 ('Customer2', 'Factory3'): 9,
 ('Customer3', 'Factory1'): 6,
 ('Customer3', 'Factory2'): 7,
 ('Customer3', 'Factory3'): 5}

In [132]:
import numpy as np
W_matrix_original = np.zeros((len(factory_locations), len(customer_regions)))

# 遍历工厂位置和客户位置，从字典中获取成本并填充矩阵
for i, factory in enumerate(factory_locations):
    for j, customer in enumerate(customer_regions):
        cost = transport_costs.get(factory, {}).get(customer, 0) # 使用 .get() 方法安全地处理字典中可能不存在的键
        W_matrix_original[i, j] = cost

In [133]:
W_matrix_original

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])